In [1]:
import numpy as np
import time
from collections import deque
from random import randint

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models import Legend, LegendItem
from bokeh.models import Range1d
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

# Serial data readout

Stateful generator for parsing the serial feed from the sensor, rebooting it at start up.

In [60]:
import enum
from typing import Any, Iterator


def parse_measurement_line(line: str) -> list[float]:
    return list((float(e) for e in line.split(',')))


# TODO: replace Any with a proper type hint
def console_parser(serial_feed: Iterator[str]) -> Iterator[Any]:
    class State(enum.Enum):
        BOOTING = enum.auto()
        WAITING_FOR_FIRST = enum.auto()
        SAMPLING = enum.auto()

    state = State.BOOTING
    
    for msg in serial_feed:
        next_state = state
        
        if state == State.BOOTING:
            if 'WHO_AM_I: 0x' in msg:
                next_state = State.WAITING_FOR_FIRST
                
        elif state == State.WAITING_FOR_FIRST:
            try:
                yield parse_measurement_line(msg)
                next_state = State.SAMPLING
            except Exception:
                pass

        elif state == State.SAMPLING:
            if msg.startswith("DBG:"):
                # print(msg)
                8==0
            else:
                try:
                    yield parse_measurement_line(msg)
                except Exception:
                    next_state = State.BOOTING

        if state != next_state:
            print(f">>>Change state to {next_state.name}")
        state = next_state

Generator that wraps the serial port as a stream of lines

In [52]:
import time
import serial


def lines_from_serial(port="/dev/ttyACM0",
                      baudrate=115200) -> Iterator[str]:
    with serial.Serial(port=port,
                       baudrate=baudrate,
                       dsrdtr=False) as sp:
        # Pulse DTR for resetting
        sp.setDTR(True)
        time.sleep(0.5)
        sp.setDTR(False)

        while True:
            read_in: bytes = sp.readline()
            line = read_in.decode().rstrip()
            yield line

Chain both generators:

In [4]:
lines = lines_from_serial()
for i, meas in enumerate(console_parser(lines)):
    print(time.time(), meas)
    if i == 10:
        break

>>>Change state to WAITING_FOR_FIRST
1719659304.666501 [-0.294, -0.153, 0.079]
>>>Change state to SAMPLING
1719659304.6749456 [-0.299, -0.168, 0.127]
1719659304.6846306 [-0.333, -0.144, 0.038]
1719659304.6949472 [-0.319, -0.185, 0.103]
1719659304.70559 [-0.324, -0.185, 0.045]
1719659304.7154558 [-0.307, -0.166, 0.043]
1719659304.7250528 [-0.293, -0.129, -0.0]
1719659304.7350404 [-0.302, -0.18, 0.079]
1719659304.745201 [-0.276, -0.182, 0.089]
1719659304.754655 [-0.3, -0.192, 0.048]
1719659304.765316 [-0.329, -0.217, 0.045]


## Receive-side sample pre-processsing

Various pre-processing functions, including the trivial identity one

In [5]:
def identity(meas: list[float]) -> list[float]:
    return meas

In [6]:
# The offsets are calculated on the "Some tests" section below
def deoffset(meas: list[float]) -> list[float]:
    return [meas[0] - off_ax, meas[1] - off_ay, meas[2] - off_az]

In [7]:
import enum
from typing import Optional, Callable


class MotionState(enum.Enum):
    AT_REST = enum.auto()
    MOVING = enum.auto()


class MotionObserver:
    
    def __init__(self, mag_reset_thd: float, window: int, state_cb: Optional[Callable]=None):
        self.mag_reset_threshold = mag_reset_thd
        self.state_cb = state_cb
        
        self.buffer = deque(maxlen=window)
        self.state = MotionState.AT_REST

    def __call__(self, new: list[float]) -> float:
        vec = np.asarray(new)
        self.buffer.append(np.linalg.norm(vec))
        figure = np.percentile(np.asarray(self.buffer), 75)
        self.magnitude_reset_criterion(figure)
        return figure

    def magnitude_reset_criterion(self, magnitude: float) -> None:
        next_state = self.state
        if self.state == MotionState.AT_REST and magnitude > self.mag_reset_threshold:
            next_state = MotionState.MOVING
        elif self.state == MotionState.MOVING and magnitude < self.mag_reset_threshold:
            next_state = MotionState.AT_REST

        if next_state != self.state and self.state_cb:
            self.state_cb(next_state.name.replace("_", " "))
        self.state = next_state

In [8]:
class OffsetState(enum.Enum):
    AT_REST_ACCUMULATING = enum.auto()
    AT_REST_IDLE = enum.auto()
    MOVING = enum.auto()


class DynamicOffsetCompensator:
    
    def __init__(self, mag_reset_thd: float, window: int, state_cb: Optional[Callable]):
        self.mag_reset_threshold = mag_reset_thd
        self.state_cb = state_cb

        self.offset = np.r_[0, 0, 0]
        self.buffer = deque(maxlen=window)
        self.state = OffsetState.AT_REST_ACCUMULATING

    def __call__(self, new: list[float]) -> list[float]:
        self.buffer.append(new)
        vecs = np.asarray(self.buffer)
        magnitudes = np.linalg.norm(vecs, axis=1)
        figure = np.percentile(magnitudes, 75)
        self.magnitude_reset_criterion(figure, vecs)
        
        return list(new[i] - self.offset[i] for i in range(len(new)))

    def magnitude_reset_criterion(self, magnitude: float, history: np.array) -> None:
        next_state = self.state
        std = history.std(axis=0).max()
        
        if self.state == OffsetState.AT_REST_ACCUMULATING:
            if magnitude > self.mag_reset_threshold:
                next_state = OffsetState.MOVING
            elif std < 0.2*self.mag_reset_threshold:
                next_state = OffsetState.AT_REST_IDLE
                self.offset = history.mean(axis=0)
                self.state_cb(next_state.name.replace("_", " ") + "!")
                
        elif self.state == OffsetState.AT_REST_IDLE:
            if magnitude > self.mag_reset_threshold:
                next_state = OffsetState.MOVING
                
        elif self.state == OffsetState.MOVING:
            if magnitude < self.mag_reset_threshold:
                next_state = OffsetState.AT_REST_ACCUMULATING

        if next_state != self.state and self.state_cb:
            self.state_cb(next_state.name.replace("_", " "))

        self.state = next_state

In [102]:
class TMeansSubtractor():
    def __init__(self, window: int):
        self.buffer = deque(maxlen=window)

    def __call__(self, new: list[float]) -> list[float]:
        vecs = np.asarray(self.buffer)
        means = np.mean(vecs, axis=0)
        self.buffer.append(new)

        return list(np.asarray(new) - means)


# Live plotting from serial

In [68]:
# Data sampling period [seconds]
sample_period = 10e-3

# Data capture buffer length [amount of samples]
depth = 250

# Live display rate [frames per second]
plot_upd_fps = 30

# data columns constructor
def new_data():
    return dict(
        t=[],
        x=[],
        y=[],
        z=[],
        mag=[],
        dt=[],
    )

In [69]:
source = ColumnDataSource(new_data())
p = figure()
p.line(source=source, x='t', y='x', legend_label="x", line_color="green")
p.line(source=source, x='t', y='y', legend_label="y", line_color="blue")
p.line(source=source, x='t', y='z', legend_label="z", line_color="red")
p.line(source=source, x='t', y='mag', legend_label="mag", line_color="black")
p.title.text = "Accelerations"
p.y_range = Range1d(-6, 6)

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [80]:
new_samples = new_data()
all_samples = []
plot_upd_thd = int(1/(sample_period*plot_upd_fps))
last = None

def set_title(state: str):
    p.title.text = state

#preprocessor = unrotated_accel
preprocessor = TMeansSubtractor(100)
#preprocessor = deoffset

postprocessor = identity
# postprocessor = DynamicOffsetCompensator(1.8, 30, set_title)


indicator = MotionObserver(1.8, 30,set_title)

# Flush source's data
source.stream(new_samples, 1)
try:
    for i, meas in enumerate(console_parser(lines_from_serial())):
        # Receive loop timing consistency observability
        now = time.time()
        dt = now - last if last is not None else sample_period
        last = now

        meas = preprocessor(meas)
        meas = postprocessor(meas)
        #mag = np.linalg.norm(meas)
        #meas.append(mag)
        meas.append(indicator(meas))
        meas.append(dt/sample_period)

        # Store data
        new_samples['t'].append(i*sample_period)
        new_samples['x'].append(meas[0])
        new_samples['y'].append(meas[1])
        new_samples['z'].append(meas[2])
        new_samples['mag'].append(meas[3])
        new_samples['dt'].append(meas[4])
        all_samples.append(meas)

        if i % plot_upd_thd == 0:
            source.stream(new_samples, depth)
            # push updates to the plot continuously using the handle
            # (interrupt the notebook kernel to stop)
            push_notebook(handle=target)

            # Refresh for next accumulation phase
            new_samples = new_data()
        
except KeyboardInterrupt:
    print("Stopped.")
    
finally:
    print(f"All samples are {len(all_samples)}")

>>>Change state to WAITING_FOR_FIRST
means nan
>>>Change state to SAMPLING
means [-0.652 -0.292  0.036]
means [-0.6665 -0.28   -0.0285]
means [-0.64833333 -0.28166667 -0.02466667]
means [-0.6475 -0.282  -0.0215]
means [-0.6366 -0.2774 -0.0182]
means [-0.63533333 -0.278      -0.01716667]
means [-0.63542857 -0.28214286 -0.01442857]
means [-0.633125 -0.27925  -0.008125]
means [-0.63       -0.28311111 -0.01144444]
means [-0.6332 -0.2807 -0.0024]
means [-0.63327273 -0.28218182 -0.01281818]
means [-0.63691667 -0.27866667 -0.00733333]
means [-0.63338462 -0.27530769 -0.00915385]
means [-0.6365     -0.27007143 -0.01092857]
means [-0.6378     -0.26793333 -0.00433333]
means [-0.6389375 -0.2675625 -0.0048125]
means [-0.63994118 -0.26652941 -0.00511765]
means [-0.64005556 -0.26961111 -0.00366667]
means [-0.64231579 -0.27252632 -0.00363158]
means [-0.64315 -0.27405 -0.00755]
means [-0.64280952 -0.27495238 -0.00695238]
means [-0.64140909 -0.27504545 -0.00818182]
means [-0.63795652 -0.27647826 -0.0073

# Velocity

In [54]:

class VelocityTracker:
    def __init__(self, relative_drag: list[float]):
        self.relative_drag = relative_drag
        self.velocity = [0.0,0.0,0.0]

    def __call__(self, new: list[float], dt: float) -> list[float]:
        for i in range(3):
            self.velocity[i] = new[i] * dt + self.velocity[i]
            new[i] = self.velocity[i]
        return new


In [55]:
velocity_source = ColumnDataSource(new_data())
p = figure()
p.line(source=velocity_source, x='t', y='x', legend_label="x", line_color="green")
p.line(source=velocity_source, x='t', y='y', legend_label="y", line_color="blue")
p.line(source=velocity_source, x='t', y='z', legend_label="z", line_color="red")
p.line(source=velocity_source, x='t', y='mag', legend_label="mag", line_color="black")
p.title.text = "Accelerations"
p.y_range = Range1d(-6, 6)

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [103]:
new_samples = new_data()
all_samples = []
plot_upd_thd = int(1/(sample_period*plot_upd_fps))
last = None

def set_title(state: str):
    p.title.text = state

#preprocessor = unrotated_accel
preprocessor = identity
#preprocessor = deoffset

# postprocessor = identity
postprocessor = DynamicOffsetCompensator(1.8, 30, set_title)

velocity_converter = VelocityTracker([0.5,0.5,0.5])


indicator = MotionObserver(1.8, 30,set_title)

# Flush source's data
velocity_source.stream(new_samples, 1)

current_velocity= {
    'x': 0,
    'y': 0,
    'z': 0
}

try:
    for i, meas in enumerate(console_parser(lines_from_serial())):
        # Receive loop timing consistency observability
        now = time.time()
        dt = now - last if last is not None else sample_period
        last = now

        meas = preprocessor(meas)
        meas = postprocessor(meas)
        meas = velocity_converter(meas,dt)
        #mag = np.linalg.norm(meas)
        #meas.append(mag)
        meas.append(indicator(meas))
        meas.append(dt/sample_period)

        current_velocity['x']

        # Store data
        new_samples['t'].append(i*sample_period)
        new_samples['x'].append(meas[0])
        new_samples['y'].append(meas[1])
        new_samples['z'].append(meas[2])
        new_samples['mag'].append(meas[3])
        new_samples['dt'].append(meas[4])
        all_samples.append(meas)

        if i % plot_upd_thd == 0:
            velocity_source.stream(new_samples, depth)
            # push updates to the plot continuously using the handle
            # (interrupt the notebook kernel to stop)
            push_notebook(handle=target)

            # Refresh for next accumulation phase
            new_samples = new_data()
        
except KeyboardInterrupt:
    print("Stopped.")
    
finally:
    print(f"All samples are {len(all_samples)}")

>>>Change state to WAITING_FOR_FIRST
>>>Change state to SAMPLING
Stopped.
All samples are 2327


# Positional Gesture Detection

In [90]:
test_value = np.array([[1,2,3],[4,5,6], [7,8,9]])
np.linalg.norm(test_value[:,[0,1]], axis=1)

array([ 2.23606798,  6.40312424, 10.63014581])

In [171]:
class Gesture(enum.Enum):
    HORIZONTAL = enum.auto()
    VERTICAL = enum.auto()
    DIAGONAL = enum.auto()


class Analysis():
    def __init__(self, threshold: float = 1.0, diagonal_threshold: float = 0.8, window: int = 20):
        self.buffer = deque(maxlen=window)
        self.threshold = threshold
        self.diagonal_threshold = diagonal_threshold
        
    def __call__(self, meas: list[float]) -> Optional[Gesture]:
        if len(meas) < 3:
            return None
        self.buffer.append(meas[0:3])
        return self.compute_gesture()
    
    def horizontal_velocity(self):
        data = np.asarray(self.buffer)
        return np.linalg.norm(data[:,[0,1]], axis=1)
        
    def vertical_velocity(self):
        data = np.asarray(self.buffer)
        return np.absolute(data[:,2])

        
    def compute_gesture(self):
        horizontal = np.quantile(self.horizontal_velocity(), 0.75)
        vertical = np.quantile(self.vertical_velocity(), 0.75)

        if horizontal > self.diagonal_threshold and vertical > self.diagonal_threshold:
            return Gesture.DIAGONAL
        elif horizontal > self.threshold:
            return Gesture.HORIZONTAL
        elif vertical > self.threshold:
            return Gesture.VERTICAL
        else:
            return None



In [184]:
def gesture_data():
    return dict(
        t= [],
        x= [],
        y= [],
        x_75percent= [],
        y_75percent= []
    )
gesture_source = ColumnDataSource(gesture_data())
p = figure()
p.line(source=gesture_source, x='t', y='x', legend_label="x", line_color="green")
p.line(source=gesture_source, x='t', y='y', legend_label="y", line_color="blue")
p.line(source=gesture_source, x='t', y='x_75percent', legend_label="x", line_color="lightgreen")
p.line(source=gesture_source, x='t', y='y_75percent', legend_label="y", line_color="lightblue")
p.title.text = "Accelerations"
p.y_range = Range1d(-6, 6)

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [186]:
import math

new_samples = gesture_data()
all_samples = []
plot_upd_thd = int(1/(sample_period*plot_upd_fps))
last = None

def set_title(state: str):
    p.title.text = state

#preprocessor = unrotated_accel
# preprocessor = identity
#preprocessor = deoffset
preprocessor = TMeansSubtractor(100)

postprocessor = identity
# postprocessor = DynamicOffsetCompensator(1.8, 30, set_title)

indicator = MotionObserver(1.8, 50,set_title)

analysis = Analysis(2.0, diagonal_threshold= 1.5, window = 30)
prev_state = None

try:
    for i, meas in enumerate(console_parser(lines_from_serial())):
        # Receive loop timing consistency observability
        now = time.time()
        dt = now - last if last is not None else sample_period
        last = now

        meas = preprocessor(meas)
        meas = postprocessor(meas)
        an = analysis(meas)
        if an != prev_state:
            print(an)
            prev_state = an

        meas.append(indicator(meas))
        meas.append(dt/sample_period)
        
        # Store data
        new_samples['t'].append(i*sample_period)
        new_samples['x'].append(np.mean(analysis.horizontal_velocity()))
        new_samples['y'].append(np.mean(analysis.vertical_velocity()))
        
        new_samples['x_75percent'].append(np.quantile(analysis.horizontal_velocity(),0.75))
        new_samples['y_75percent'].append(np.quantile(analysis.vertical_velocity(),0.75))
        all_samples.append(meas)

        if i % plot_upd_thd == 0:
            gesture_source.stream(new_samples, depth)
            # push updates to the plot continuously using the handle
            # (interrupt the notebook kernel to stop)
            push_notebook(handle=target)

            # Refresh for next accumulation phase
            new_samples = new_data()
        
except KeyboardInterrupt:
    print("Stopped.")
    
finally:
    print(f"All samples are {len(all_samples)}")

>>>Change state to WAITING_FOR_FIRST
>>>Change state to SAMPLING
All samples are 1


KeyError: 'x_75percent'

# Offline data analysis

In [ ]:
all_array = np.array(all_samples)

t = np.r_[:all_array.shape[0]] * sample_period
t

p = figure()
p.line(t, all_array[:, 0], legend_label="x", line_color="green")
p.line(t, all_array[:, 1], legend_label="y", line_color="blue")
p.line(t, all_array[:, 2], legend_label="z", line_color="red")
p.line(t, all_array[:, 3], legend_label="mag", line_color="black")

show(p)

### Some tests

In [ ]:
off_ax, off_ay, off_az = all_array.mean(axis=0)[:3]
(off_ax, off_ay, off_az)